# Transfer leraning with keras: CNN for binary classification

Odkud jsem čerpala za poslední dva týdny: 

https://machinelearningmastery.com/how-to-use-transfer-learning-when-developing-convolutional-neural-network-models/

https://www.pyimagesearch.com/2018/12/24/how-to-use-keras-fit-and-fit_generator-a-hands-on-tutorial/

https://www.youtube.com/playlist?list=PLZbbT5o_s2xrwRnXk_yCPtnqqo4_u2YGL

https://keras.io/applications/


In [1]:
#import knihoven

In [3]:
import keras
from keras.layers.core import Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import load_img,img_to_array

Using TensorFlow backend.


In [3]:
#path k trénovacím a testovacím datům

In [4]:
train_path='/train'
test_path= '/test'

In [5]:
#import obrázků, jejich pre-processing a tvorba tzv. batches pro trénování a testování

In [6]:
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(224,224), 
                                                         classes=['jablka','rajcata'], batch_size=10)
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224), 
                                                        classes=['jablka','rajcata'],batch_size=4)

Found 40 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [7]:
#import modelu VGG16 bez poslední vrstvy, která predikuje obrázky do jedné z tisíci tříd
#specifikace vstupní vrstvy na rozměr obrázku (224, 224) jaký jsme použili i k pre-processingu o buňku výš

In [8]:
vgg16_model = VGG16(include_top=False, input_shape=(224, 224, 3))

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
#poslední vrstva ve vgg16_model, je nyní vrstva s (MaxPooling2D, (None, 7, 7, 512))
#proto definujeme flat1 vrstvu, která rozměr 7x7x512 rozvine do vektoru s dimenzí 25088
#z této flatten vrstvy budeme rovnou predikovat obrázek do jedné ze tříd jablka vs. hrušky
#definujeme output vrstvu o dimenzi 2

In [10]:
flat1 = Flatten()(vgg16_model.output)
output = Dense(2, activation='softmax')(flat1)

In [11]:
#vytvoříme nový klasifikátor clf nasazením vrstev vgg16_model modelu na vrstvu flat1 a output

In [12]:
clf = Model(inputs=vgg16_model.inputs, outputs=output)

In [13]:
#protože v clf chceme zachovat informaci o vahách, které ve vgg16_model už byly 
#(někdo tento model trénoval na obrovském množství dat a tím váhy dobře nastavil),
#všechny vrstvy z původního vgg16_model zafixujeme, budeme trénovat pouze naše nové vrstvy flat1 a output

In [14]:
for layer in clf.layers:
    layer.trainable = False
for layer in clf.layers[-2:]:
    layer.trainable = True

In [15]:
#sumarizace modelu clf, všechny jeho vrstvy, počty parametrů (trainable, Non-trainable)

In [16]:
clf.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [17]:
#kompilování modelu, výběr optimizer funkce, typu loss funkce a metriky

In [18]:
clf.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics = ['accuracy'] )

In [19]:
#fitování modelu clf na naše data

In [20]:
clf.fit_generator(train_batches, steps_per_epoch=4, validation_data=test_batches, validation_steps=10, epochs=3)

Instructions for updating:
Use tf.cast instead.
Epoch 1/3
4/4 [==============================] - 30s 7s/step - loss: 3.4076 - acc: 0.6500 - val_loss: 0.3270 - val_acc: 0.9500
Epoch 2/3
4/4 [==============================] - 31s 8s/step - loss: 0.8323 - acc: 0.9250 - val_loss: 1.0467 - val_acc: 0.9000
Epoch 3/3
4/4 [==============================] - 33s 8s/step - loss: 0.4040 - acc: 0.9750 - val_loss: 1.7759 - val_acc: 0.8500


In [22]:
#natrénovaný model clf nad obrázky z listu ingrediencí uloží do JSONu
#pro predikci v cooking app se importuje podobně, jako jsme sem na začátku importovali VGG16 model
#předložený obrázek se naformátuje a oklasifikuje funkcí clf.predict()

In [5]:
from keras.models import model_from_json
import numpy as np
import os

In [24]:
model_json = clf.to_json()

In [25]:
with open("clf.json", "w") as json_file:
    json_file.write(model_json)

In [26]:
clf.save_weights("model.h5")

In [27]:
#a jak to bude vypadat ve funkci pro cooking-app?

In [6]:
def klasifikace(path):
    classes = ['jablka','rajcata']
    im=load_img(path, target_size=(224, 224))
    im=img_to_array(im)
    im = im.reshape((1, im.shape[0], im.shape[1], im.shape[2]))
    prepared_images = preprocess_input(im)
    json_file = open('clf.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights("model.h5")
    loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    predikce = loaded_model.predict(im).tolist()[0]
    index= predikce.index(max(predikce))
    output = classes[index]
    return output


In [8]:
klasifikace('manzana-large.jpg')

'jablka'

In [9]:
klasifikace('rajce.jpeg')

'rajcata'